# Week 5 - Technical Assignment: Fine Tuning using LoRA and PEFT
This notebook covers the fine-tuning of an instruction-following LLM using LoRA and PEFT techniques. We will go through all the required steps, from loading the model to uploading it to Hugging Face.

## 1. Install Required Libraries

In [ ]:

!pip install transformers datasets peft bitsandbytes accelerate huggingface_hub
!pip install sentencepiece  # Required for certain models


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

## 2. Load the Dataset from Hugging Face

In [ ]:

from datasets import load_dataset

# Load a sample dataset from Hugging Face
dataset = load_dataset("imdb", split="train[:1%]")
print(dataset[0])


{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

## 3. Create Bitsandbytes Configuration

In [ ]:

import bitsandbytes as bnb

# No specific configuration needed for basic quantization in this example
print("Bitsandbytes loaded successfully.")


ERROR:bitsandbytes.cextension:Could not load bitsandbytes native library: /lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.32' not found (required by /usr/local/lib/python3.11/dist-packages/bitsandbytes/libbitsandbytes_cpu.so)
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/bitsandbytes/cextension.py", line 85, in <module>
    lib = get_native_library()
          ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/bitsandbytes/cextension.py", line 72, in get_native_library
    dll = ct.cdll.LoadLibrary(str(binary_path))
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 454, in LoadLibrary
    return self._dlltype(name)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 376, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: /lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4

Bitsandbytes loaded successfully.


In [ ]:
import torch
print(torch.cuda.is_available())

False


In [ ]:
!pip uninstall -y bitsandbytes
!pip install -U bitsandbytes
!pip install bitsandbytes-cuda117

Found existing installation: bitsandbytes 0.45.2
Uninstalling bitsandbytes-0.45.2:
  Successfully uninstalled bitsandbytes-0.45.2
  Using cached bitsandbytes-0.45.2-py3-none-manylinux_2_24_x86_64.whl.metadata (5.8 kB)
Using cached bitsandbytes-0.45.2-py3-none-manylinux_2_24_x86_64.whl (69.7 MB)
  Using cached bitsandbytes_cuda117-0.26.0.post2-py3-none-any.whl.metadata (6.3 kB)
Using cached bitsandbytes_cuda117-0.26.0.post2-py3-none-any.whl (4.3 MB)


In [ ]:
import bitsandbytes as bnb
import torch

print(bnb.__version__)
print("CUDA available:", torch.cuda.is_available())


OSError: libcudart.so.11.0: cannot open shared object file: No such file or directory

## 4. Load the Pre-Trained Model (LLaMA 1.1B or Similar)

In [ ]:

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "tiiuae/falcon-7b-instruct"  # Example of a smaller model for testing
tokenizer = AutoTokenizer.from_pretrained(model_name)

quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto"
)

print("Model and tokenizer loaded successfully.")



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model and tokenizer loaded successfully.


In [ ]:

from datasets import load_dataset

# Load a sample dataset from Hugging Face
dataset = load_dataset("imdb", split="train[:1%]")
print(dataset[0])


{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [ ]:

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})


## 5. Tokenization of the Dataset

In [ ]:

from transformers import DataCollatorWithPadding

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128, return_tensors="pt")

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Use a data collator that handles padding and attention masks
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

print(tokenized_dataset[0])


{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

## 6. Zero-Shot Inference Test

In [ ]:

inputs = tokenizer("What is the capital of France?", return_tensors="pt").to('cuda')
outputs = model.generate(inputs['input_ids'], max_length=20)
print(tokenizer.decode(outputs[0]))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


What is the capital of France?
The capital of France is Paris.<|endoftext|>


## 7. Pre-process the Dataset

In [ ]:

# Prepare the dataset for training
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset.set_format("torch")


## 8. Prepare the Model for QLoRA

In [ ]:

# Enable gradient checkpointing (no need to move model to 'cuda' manually)
model.gradient_checkpointing_enable()

# The model is already on the correct device when using 8-bit quantization
print("Gradient checkpointing enabled. Model is already on the correct device.")



Gradient checkpointing enabled. Model is already on the correct device.


## 9. Set Up PEFT for Fine-Tuning

In [ ]:
for name, module in model.named_modules():
    print(name)




transformer
transformer.word_embeddings
transformer.h
transformer.h.0
transformer.h.0.self_attention
transformer.h.0.self_attention.query_key_value
transformer.h.0.self_attention.dense
transformer.h.0.self_attention.attention_dropout
transformer.h.0.self_attention.rotary_emb
transformer.h.0.mlp
transformer.h.0.mlp.dense_h_to_4h
transformer.h.0.mlp.act
transformer.h.0.mlp.dense_4h_to_h
transformer.h.0.input_layernorm
transformer.h.1
transformer.h.1.self_attention
transformer.h.1.self_attention.query_key_value
transformer.h.1.self_attention.dense
transformer.h.1.self_attention.attention_dropout
transformer.h.1.self_attention.rotary_emb
transformer.h.1.mlp
transformer.h.1.mlp.dense_h_to_4h
transformer.h.1.mlp.act
transformer.h.1.mlp.dense_4h_to_h
transformer.h.1.input_layernorm
transformer.h.2
transformer.h.2.self_attention
transformer.h.2.self_attention.query_key_value
transformer.h.2.self_attention.dense
transformer.h.2.self_attention.attention_dropout
transformer.h.2.self_attention.ro

In [ ]:
from peft import LoraConfig, get_peft_model

# Update target modules based on model inspection
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["self_attention.query_key_value"],  # Update with actual module names
    lora_dropout=0.05,
    bias="none"
)

model = get_peft_model(model, lora_config)
print("PEFT model configured.")


PEFT model configured.


In [ ]:
print(tokenized_dataset.column_names)
print(tokenized_dataset[0])


['label', 'input_ids', 'attention_mask']
{'label': tensor(0), 'input_ids': tensor([   52, 28694,   295,  4582,   319,  3106, 42353,    24, 38845, 48954,
          427,   491,  2028,  2946,   875,   275,   455,   248, 21681,   325,
        12380,   334,   635,   334,   398,   772,  4171,   272,   204,  3954,
           34,    25,   295,   614,  3262,   325,   388,   772,   334,   398,
        25208,   431,   493,    25,    62,    25, 21359,   565,   334,  1779,
         3506,   271,  3464,   414,  2020,    23,  4859,  1003,   241,  4838,
          275,  7593,  3558,   204,    13,  3079, 49698,   470,    13,   295,
         1037,   618,   271,   760,   414,   312,  2838,  8275,  1162,   204,
         5285,  1162,   204,  2541,   487,  8965,   304, 20813,  1111,   241,
         2115, 20132, 11125,  3457,  5200, 51037,   569,  4045,   271,  1730,
         2096,   674,   418,   544,  1063,    25,   529,  2057,   674,  4045,
          271,  1966,   573, 23901,   482,   271,  1591,   596,  32

In [ ]:
def preprocess_for_clm(example):
    # Ensure input_ids and labels are properly set
    example["input_ids"] = example["input_ids"]
    example["labels"] = example["input_ids"]
    return example

# Apply the preprocessing to fix the dataset
tokenized_dataset = tokenized_dataset.map(preprocess_for_clm, batched=True)
print(tokenized_dataset[0])




Map:   0%|          | 0/250 [00:00<?, ? examples/s]

{'label': tensor(0), 'input_ids': tensor([   52, 28694,   295,  4582,   319,  3106, 42353,    24, 38845, 48954,
          427,   491,  2028,  2946,   875,   275,   455,   248, 21681,   325,
        12380,   334,   635,   334,   398,   772,  4171,   272,   204,  3954,
           34,    25,   295,   614,  3262,   325,   388,   772,   334,   398,
        25208,   431,   493,    25,    62,    25, 21359,   565,   334,  1779,
         3506,   271,  3464,   414,  2020,    23,  4859,  1003,   241,  4838,
          275,  7593,  3558,   204,    13,  3079, 49698,   470,    13,   295,
         1037,   618,   271,   760,   414,   312,  2838,  8275,  1162,   204,
         5285,  1162,   204,  2541,   487,  8965,   304, 20813,  1111,   241,
         2115, 20132, 11125,  3457,  5200, 51037,   569,  4045,   271,  1730,
         2096,   674,   418,   544,  1063,    25,   529,  2057,   674,  4045,
          271,  1966,   573, 23901,   482,   271,  1591,   596,  3229,   275,
        15268,   313,   637,  

In [ ]:
model.config.use_cache = False


In [ ]:
print(tokenized_dataset.column_names)
print("input_ids:", tokenized_dataset[0]["input_ids"])
print("labels:", tokenized_dataset[0]["labels"])


['label', 'input_ids', 'attention_mask', 'labels']
input_ids: tensor([   52, 28694,   295,  4582,   319,  3106, 42353,    24, 38845, 48954,
          427,   491,  2028,  2946,   875,   275,   455,   248, 21681,   325,
        12380,   334,   635,   334,   398,   772,  4171,   272,   204,  3954,
           34,    25,   295,   614,  3262,   325,   388,   772,   334,   398,
        25208,   431,   493,    25,    62,    25, 21359,   565,   334,  1779,
         3506,   271,  3464,   414,  2020,    23,  4859,  1003,   241,  4838,
          275,  7593,  3558,   204,    13,  3079, 49698,   470,    13,   295,
         1037,   618,   271,   760,   414,   312,  2838,  8275,  1162,   204,
         5285,  1162,   204,  2541,   487,  8965,   304, 20813,  1111,   241,
         2115, 20132, 11125,  3457,  5200, 51037,   569,  4045,   271,  1730,
         2096,   674,   418,   544,  1063,    25,   529,  2057,   674,  4045,
          271,  1966,   573, 23901,   482,   271,  1591,   596,  3229,   275,
  

In [ ]:
model.train()


PeftModel(
  (base_model): LoraModel(
    (model): FalconForCausalLM(
      (transformer): FalconModel(
        (word_embeddings): Embedding(65024, 4544)
        (h): ModuleList(
          (0-31): 32 x FalconDecoderLayer(
            (self_attention): FalconAttention(
              (query_key_value): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=4544, out_features=4672, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4544, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4672, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
             

In [ ]:
for name, param in model.named_parameters():
    print(f"{name}: dtype={param.dtype}, requires_grad={param.requires_grad}")



base_model.model.transformer.word_embeddings.weight: dtype=torch.float16, requires_grad=False
base_model.model.transformer.h.0.self_attention.query_key_value.base_layer.weight: dtype=torch.int8, requires_grad=False
base_model.model.transformer.h.0.self_attention.query_key_value.lora_A.default.weight: dtype=torch.float32, requires_grad=True
base_model.model.transformer.h.0.self_attention.query_key_value.lora_B.default.weight: dtype=torch.float32, requires_grad=True
base_model.model.transformer.h.0.self_attention.dense.weight: dtype=torch.int8, requires_grad=False
base_model.model.transformer.h.0.mlp.dense_h_to_4h.weight: dtype=torch.int8, requires_grad=False
base_model.model.transformer.h.0.mlp.dense_4h_to_h.weight: dtype=torch.int8, requires_grad=False
base_model.model.transformer.h.0.input_layernorm.weight: dtype=torch.float16, requires_grad=False
base_model.model.transformer.h.0.input_layernorm.bias: dtype=torch.float16, requires_grad=False
base_model.model.transformer.h.1.self_atten

In [ ]:
for name, param in model.named_parameters():
    if param.dtype in [torch.float16, torch.float32, torch.bfloat16]:  # Only apply to floating-point tensors
        param.requires_grad = True
    else:
        print(f"Skipping non-float parameter: {name} with dtype {param.dtype}")


Skipping non-float parameter: base_model.model.transformer.h.0.self_attention.query_key_value.base_layer.weight with dtype torch.int8
Skipping non-float parameter: base_model.model.transformer.h.0.self_attention.dense.weight with dtype torch.int8
Skipping non-float parameter: base_model.model.transformer.h.0.mlp.dense_h_to_4h.weight with dtype torch.int8
Skipping non-float parameter: base_model.model.transformer.h.0.mlp.dense_4h_to_h.weight with dtype torch.int8
Skipping non-float parameter: base_model.model.transformer.h.1.self_attention.query_key_value.base_layer.weight with dtype torch.int8
Skipping non-float parameter: base_model.model.transformer.h.1.self_attention.dense.weight with dtype torch.int8
Skipping non-float parameter: base_model.model.transformer.h.1.mlp.dense_h_to_4h.weight with dtype torch.int8
Skipping non-float parameter: base_model.model.transformer.h.1.mlp.dense_4h_to_h.weight with dtype torch.int8
Skipping non-float parameter: base_model.model.transformer.h.2.sel

In [ ]:
sample = next(iter(tokenized_dataset))
print("input_ids:", sample["input_ids"])
print("labels:", sample["labels"])


input_ids: tensor([   52, 28694,   295,  4582,   319,  3106, 42353,    24, 38845, 48954,
          427,   491,  2028,  2946,   875,   275,   455,   248, 21681,   325,
        12380,   334,   635,   334,   398,   772,  4171,   272,   204,  3954,
           34,    25,   295,   614,  3262,   325,   388,   772,   334,   398,
        25208,   431,   493,    25,    62,    25, 21359,   565,   334,  1779,
         3506,   271,  3464,   414,  2020,    23,  4859,  1003,   241,  4838,
          275,  7593,  3558,   204,    13,  3079, 49698,   470,    13,   295,
         1037,   618,   271,   760,   414,   312,  2838,  8275,  1162,   204,
         5285,  1162,   204,  2541,   487,  8965,   304, 20813,  1111,   241,
         2115, 20132, 11125,  3457,  5200, 51037,   569,  4045,   271,  1730,
         2096,   674,   418,   544,  1063,    25,   529,  2057,   674,  4045,
          271,  1966,   573, 23901,   482,   271,  1591,   596,  3229,   275,
        15268,   313,   637,   248,  3608,  3260,  94

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["self_attention.query_key_value"],  # Use valid module names from the model
    lora_dropout=0.05,
    bias="none"
)

model = get_peft_model(model, lora_config)
model.train()

for name, param in model.named_parameters():
    print(f"{name}: dtype={param.dtype}, requires_grad={param.requires_grad}")


base_model.model.base_model.model.transformer.word_embeddings.weight: dtype=torch.float16, requires_grad=False
base_model.model.base_model.model.transformer.h.0.self_attention.query_key_value.base_layer.weight: dtype=torch.int8, requires_grad=False
base_model.model.base_model.model.transformer.h.0.self_attention.query_key_value.lora_A.default.weight: dtype=torch.float32, requires_grad=True
base_model.model.base_model.model.transformer.h.0.self_attention.query_key_value.lora_B.default.weight: dtype=torch.float32, requires_grad=True
base_model.model.base_model.model.transformer.h.0.self_attention.dense.weight: dtype=torch.int8, requires_grad=False
base_model.model.base_model.model.transformer.h.0.mlp.dense_h_to_4h.weight: dtype=torch.int8, requires_grad=False
base_model.model.base_model.model.transformer.h.0.mlp.dense_4h_to_h.weight: dtype=torch.int8, requires_grad=False
base_model.model.base_model.model.transformer.h.0.input_layernorm.weight: dtype=torch.float16, requires_grad=False
bas

In [ ]:
from transformers import DataCollatorForLanguageModeling

# Create a data collator for causal language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Set mlm (masked language modeling) to False for causal LM
)



In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # For causal language modeling
)


In [ ]:
from transformers import TrainingArguments, Trainer

model.config.use_cache = False

training_args = TrainingArguments(
    output_dir="./results",
    max_steps=500,
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=50,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    save_steps=100,
    save_total_limit=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    data_collator=data_collator,  # Use the custom data collator
)

trainer.train()


ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['label']

In [ ]:
from transformers import TrainingArguments, Trainer

model.config.use_cache = False

training_args = TrainingArguments(
    output_dir="./results",
    max_steps=500,
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=50,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    save_steps=100,
    save_total_limit=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    data_collator=data_collator,  # Use the custom data collator
)

trainer.train()


ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['label']

## 10. Train PEFT Adapter

## 11. Qualitative Evaluation

In [ ]:

inputs = tokenizer("Explain the theory of relativity.", return_tensors="pt").to('cuda')
outputs = model.generate(inputs['input_ids'], max_length=50)
print(tokenizer.decode(outputs[0]))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Explain the theory of relativity.
The theory of relativity, proposed by Albert Einstein, consists of two separate theories: the special theory of relativity and the general theory of relativity. The special theory of relativity, introduced in 


## 12. Quantitative Evaluation using ROUGE

In [ ]:

from datasets import load_metric

rouge = load_metric("rouge")
predictions = ["This is the predicted text."]
references = ["This is the reference text."]
results = rouge.compute(predictions=predictions, references=references)
print(results)


ImportError: cannot import name 'load_metric' from 'datasets' (/usr/local/lib/python3.11/dist-packages/datasets/__init__.py)

## 13. Save and Upload the Model to Hugging Face

In [ ]:

model.push_to_hub("your-huggingface-username/your-model-name")


HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/repos/create (Request ID: Root=1-67b598f2-6f7e05e83cf027bb22ffa38f;8bb75579-8471-4cfe-9b38-d427c36a08bd)

Invalid username or password.

## 14. Capture and Document Results

In [ ]:

# Add screenshots and outputs as needed to your notebook before submission.
print("Assignment completed!")


Assignment completed!
